In [2]:
from utils.log import getLogger
import pandas as pd
from model.db import DB_ENGINE
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import pickle
import os

logger = getLogger('predict')

2018-12-20 14:45:32,614 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


[2018-12-20 14:45:32] INFO    | sqlalchemy.engine.base.Engine: SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2018-12-20 14:45:32,618 INFO sqlalchemy.engine.base.Engine ()


[2018-12-20 14:45:32] INFO    | sqlalchemy.engine.base.Engine: ()


2018-12-20 14:45:32,624 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

[2018-12-20 14:45:32] INFO    | sqlalchemy.engine.base.Engine: ()


2018-12-20 14:45:32,671 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("rawcontents")


[2018-12-20 14:45:32] INFO    | sqlalchemy.engine.base.Engine: PRAGMA table_info("rawcontents")


2018-12-20 14:45:32,675 INFO sqlalchemy.engine.base.Engine ()


[2018-12-20 14:45:32] INFO    | sqlalchemy.engine.base.Engine: ()


In [48]:
labeled = pd.read_sql(
    'SELECT rid, tag, vector FROM rawcontents WHERE assure>0.5',
    DB_ENGINE
)
labeled.head()

2018-12-20 15:28:36,290 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("SELECT rid, tag, vector FROM rawcontents WHERE assure>0.5")


[2018-12-20 15:28:36] INFO    | sqlalchemy.engine.base.Engine: PRAGMA table_info("SELECT rid, tag, vector FROM rawcontents WHERE assure>0.5")


2018-12-20 15:28:36,321 INFO sqlalchemy.engine.base.Engine ()


[2018-12-20 15:28:36] INFO    | sqlalchemy.engine.base.Engine: ()


2018-12-20 15:28:36,328 INFO sqlalchemy.engine.base.Engine SELECT rid, tag, vector FROM rawcontents WHERE assure>0.5


[2018-12-20 15:28:36] INFO    | sqlalchemy.engine.base.Engine: SELECT rid, tag, vector FROM rawcontents WHERE assure>0.5


2018-12-20 15:28:36,332 INFO sqlalchemy.engine.base.Engine ()


[2018-12-20 15:28:36] INFO    | sqlalchemy.engine.base.Engine: ()


,rid,tag,vector
0,1,1.0,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
1,2,1.0,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
2,3,0.0,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
3,4,1.0,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
4,5,1.0,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...


In [49]:
labeled.describe()

,rid,tag
count,1.018510e+05,101851.000000
mean,5.872414e+05,0.972538
std,4.706109e+05,0.163425
min,1.000000e+00,0.000000
25%,1.803830e+05,1.000000
50%,4.275950e+05,1.000000
75%,1.011050e+06,1.000000
max,1.587761e+06,1.000000


In [50]:
train = labeled.groupby('vector')['tag'].mean()

In [51]:
X = list(pd.Series(train.index).apply(pickle.loads))
y = list(train.values)

In [60]:
with open('train_data_X.bin', 'wb') as f:
    pickle.dump(X, f)

with open('train_data_y.bin', 'wb') as f:
    pickle.dump(y, f)

In [3]:
with open('train_data_X.bin', 'rb') as f:
    X = pickle.load(f)

with open('train_data_y.bin', 'rb') as f:
    y = pickle.load(f)

In [4]:
from sklearn import ensemble, svm

In [52]:
# clf = SGDClassifier(
#     random_state=ord(os.urandom(1)),
#     max_iter=512,
#     tol=1e-3,
#     penalty='elasticnet',
#     loss='modified_huber',
#     fit_intercept=False
# )
clf = svm.SVC(kernel='rbf', gamma='scale')

In [12]:
scores = cross_val_score(clf, X, y, cv=3)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.99 (+/- 0.00)


In [53]:
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## predict

In [7]:
from scipy import stats

In [ ]:
test = pd.read_sql('SELECT rid, vector FROM rawcontents', DB_ENGINE)
test.head()

In [55]:
X_test = list(test['vector'].apply(pickle.loads))

In [56]:
predict_tag = clf.predict(X_test)

In [60]:
predict_tag.mean()

0.08984375

In [61]:
result = pd.DataFrame(data={
    'rid': test['rid'],
    'content': test['content'],
    'tag': predict_tag
})
result.head()

,rid,content,tag
0,86,QC商城就是运用了区块链技术融合线上商城线下门店的社交新零售平台\n \n 他是一个将虚拟经...,0.0
1,87,《我不是药神》上映以来好评如潮，票房大涨，是一部叫好又叫座的佳作，不仅豆瓣评分9.0，更被人...,0.0
2,100,2018/7/19:这周我很少给单子。因为自己事情多，今天弄下操作计划，先讲讲黄金，这两天我...,0.0
3,101,这好像和区块链没啥关系，是外汇方面的吧？,1.0
4,102,@真心交人诚心办事 2018-07-19 09:03:48\n \n 这好像和区块链没啥关系...,0.0


In [67]:
result.describe()

,rid,tag
count,2048.000000,2048.000000
mean,1171.120605,0.089844
std,619.894848,0.286028
min,86.000000,0.000000
25%,631.500000,0.000000
50%,1170.500000,0.000000
75%,1711.250000,0.000000
max,2237.000000,1.000000


In [10]:
proba = clf.predict_proba(X_test)
entropies = stats.distributions.entropy(proba.T, base=2)

AttributeError: predict_proba is not available when  probability=False

In [ ]:
result = pd.DataFrame(data={
    'rid': test['rid'],
    'content': test['content'],
    'entropy': entropies,
    'tag': predict_tag
})
result.head()

In [ ]:
pd.value_counts(pd.cut(entropies, np.linspace(0,1,11)))

In [ ]:
result.sort_values('entropy', ascending=False)